In [47]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import scipy.stats as stats

from pandas.api.types import CategoricalDtype

from statsmodels.miscmodels.ordinal_model import OrderedModel
import statsmodels.api as sm

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [48]:
house_data = pd.read_csv('data/kc_house_data.csv')

In [207]:
test_data = house_data[['id', 'bedrooms', 'bathrooms', 'sqft_living', 'floors',
                        'condition','grade', 'yr_built', 'zipcode', 'price'
                       ]].copy()

# Drop id nulls and duplicates 
test_data.drop_duplicates(subset= 'id', keep='first', inplace=True, ignore_index = True)


# bedrooms drop all greater than 6
test_data.drop(test_data.index[test_data['bedrooms'] >= 7], inplace = True)


#bathrooms remove all bathroom values less than 1 , greater than 4 
test_data = test_data[test_data.bathrooms >= 1.0]
test_data = test_data[test_data.bathrooms <= 4.0]
test_data['bathrooms'] = test_data['bathrooms'].dropna()

# Cast grade column as ordinal 
cat_type = CategoricalDtype(categories=['4 Low', '5 Fair', '6 Low Average', '7 Average',
                                        '8 Good','9 Better', '10 Very Good', '11 Excellent',
                                        '12 Luxury', '13 Mansion'], ordered=True)
test_data.grade = test_data['grade'].astype(cat_type)

# sqft_living drop above 5000
test_data = test_data[test_data.sqft_living < 5000]


# May potentially reduce range of year built -- cutoff around 1920

# Price drop outliers 
test_data = test_data[test_data.price < 2000000]
test_data['price'] = test_data['price'].map(lambda x: np.log(x))   


# Waterfront drop nulls 
# test_data['waterfront'] = test_data['waterfront'].isna().drop()
# test_data.dropna(axis = 0, subset = 'waterfront', inplace = True)
# DataFrame.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
# **************************************************************************************
# ABOVE IS GOOD ... except for waterfront... removed waterfront from subset







test_data[nom_features].shape

(20831, 7)

In [195]:
ord_cat_selector = ['condition', 'grade']
cat_subset = test_data[ord_cat_selector]
cat_subset

ordered_condition = ['Poor', 'Fair', 'Average', 'Good', 'Very Good']
ordered_grade = ['4 Low', '5 Fair', '6 Low Average', '7 Average','8 Good',
                 '9 Better', '10 Very Good', '11 Excellent','12 Luxury', 
                 '13 Mansion']

o_enc = OrdinalEncoder(categories = [ordered_condition, ordered_grade])
o_enc.fit(cat_subset)

X_subset = pd.DataFrame(o_enc.transform(cat_subset),
                        columns = cat_subset.columns)
X_subset.shape

(20831, 2)

In [256]:
# pd.plotting.scatter_matrix(test_data[['bedrooms', 'bathrooms', 'sqft_living', 'floors',
#                         'condition','grade', 'yr_built', 'zipcode', 'price']], figsize=(10,12));

In [255]:
# test_data.waterfront

In [37]:
lr = LinearRegression()
lr

LinearRegression()

In [252]:
nom_features = ['bedrooms', 'bathrooms', 'sqft_living', 'floors', 'yr_built', 'zipcode', 'price']
ord_features = X_subset
X_ = test_data[nom_features].join(X_subset, how = 'inner')
X = X_.drop(columns = 'price')
y = X_['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [253]:
# print(len(X_train))
# print(len(X_test))
# print(len(X_train))
# print(len(X_test.values().isnull()))


In [267]:
# sns.scatterplot(X, y)

In [254]:
lr.fit(X_train, y_train)

LinearRegression()

In [269]:
lr.score(X_train, y_train)

0.49283402825765665

In [257]:
lr.coef_

array([-0.07292926,  0.12980752,  0.00040156,  0.12347285, -0.00489866,
        0.00029845, -0.0098249 ,  0.00173025])

In [258]:
X.columns

Index(['bedrooms', 'bathrooms', 'sqft_living', 'floors', 'yr_built', 'zipcode',
       'condition', 'grade'],
      dtype='object')

In [259]:
# This gets the predictions of model on test set.
y_pred = lr.predict(X_test)
#get first ten elements of prediction.
y_pred[0:10]

array([12.98852467, 13.98882329, 12.80370712, 13.01740916, 13.24573092,
       12.81989677, 13.0420421 , 12.8032616 , 12.89573338, 12.77999688])

In [260]:
MAE = mean_absolute_error(y_pred, y_test)
MAE

0.27965779168037963

In [261]:
RMSE = np.sqrt(mean_squared_error(y_pred, y_test))
RMSE

0.3472449166951404

In [265]:
# lr.summary()

In [ ]:
"""
sqft, zipcode, grade -- median price 


"""